# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Batch Processing** </center>
---

**Date**: October, 2025

**Student Name**:

**Professor**: Pablo Camarillo Ramirez

# Introduction

OrangeStone wishes to enter the House Market, and has access to a database that is updated monthly with information pertaining to global house purchases. They want to use the most of these information to make decisions on the real state they buy.

# Dataset

The dataset they have access to is https://www.kaggle.com/datasets/mohankrishnathalla/global-house-purchase-decision-dataset


# Transformations and Actions

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML: Logistic Regression") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/26 00:57:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from regalado_floriano.spark_utils import SparkUtils 

In [3]:
houses_schema = SparkUtils.generate_schema(
 
    (("property_id","int"),
("country","string"),
("city","string"),
("property_type","string"),
("furnishing_status","string"),
("property_size_sqft","int"),
("price","int"),
("constructed_year","int"),
("previous_owners","int"),
("rooms","int"),
("bathrooms","int"),
("garage","bool"),
("garden","bool"),
("crime_cases_reported","int"),
("legal_cases_on_property","bool"),
("customer_salary","int"),
("loan_amount","int"),
("loan_tenure_years","int"),
("monthly_expenses","int"),
("down_payment","int"),
("emi_to_income_ratio","float"),
("satisfaction_score","int"),
("neighbourhood_rating","int"),
("connectivity_score","int"),
("decision","bool")
    )
)

In [4]:
house_df = spark.read \
                .option("header", "true") \
                .schema(houses_schema) \
                .csv("/opt/spark/work-dir/data/house_purchases")
 

In [16]:
house_df= house_df.na.fill(False)
 

In [17]:
#in order to prepare for ease of analysis, all categorical data will be put in it's own frame

In [18]:
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id

In [19]:
categories = "country city property_type furnishing_status".split()

In [20]:
_localMap = SparkUtils.generate_keyed_distinct_column(house_df)

categoricalTables =   {
     key:     _localMap(key) for key in categories
 }

In [21]:
from functools import reduce

In [35]:
id_house = house_df
for cat in categories:
    cur_df = categoricalTables[cat]
    id_house = SparkUtils.replace_column_for_key(id_house)(cur_df)(cat)

    

In [41]:
house_details = id_house.select("property_id","legal_cases_on_property", 
                                "previous_owners","rooms","bathrooms", "garage","garden","crime_cases_reported","legal_cases_on_property",
                               "neighbourhood_rating", "satisfaction_score", "property_size_sqft", "price")

house_no_h_details = id_house.drop("property_size_sqft", "price","legal_cases_on_property", 
                                "previous_owners","rooms","bathrooms", "garage","garden","crime_cases_reported","legal_cases_on_property",
                               "neighbourhood_rating", "satisfaction_score")


In [42]:
loan_details = id_house.select( 
            "property_id",
            "loan_amount","loan_tenure_years","down_payment"
                              )

main_house = house_no_h_details.drop( "property_id",
            "loan_amount","loan_tenure_years","down_payment" )


In [43]:
main_house.show()

+----------------+---------------+----------------+-------------------+------------------+--------+----------+-------+----------------+--------------------+
|constructed_year|customer_salary|monthly_expenses|emi_to_income_ratio|connectivity_score|decision|country_id|city_id|property_type_id|furnishing_status_id|
+----------------+---------------+----------------+-------------------+------------------+--------+----------+-------+----------------+--------------------+
|            1989|          10745|            6545|               0.16|                 6|   false|         0|     27|               4|                   1|
|            1990|          16970|            8605|               0.08|                 2|   false|         9|      0|               1|                   1|
|            2019|          21914|            2510|               0.09|                 1|   false|         9|     28|               4|                   1|
|            2008|          17980|            8805|       

## Transformations

# Persistence Data

# DAG